Решить дифференциальное уравнение методом сеток.

$
\frac{\partial u}{\partial t} = \frac{\partial}{\partial x}
\left((x + 3) \frac{\partial u}{\partial x} \right) - xu + f(x, t)
$

$u(x, 0) = \phi (x), ~ 0 \le x\le1$

$\frac{\partial u}{\partial x} |_{x=0} = \alpha(t)$

$u(1, t) = \beta (t), ~ 0\le t\le 0.1$

Тестирование алгоритма для $u(x, t) = x^3 + t^3$

In [31]:
import pandas as pd
import numpy as np

In [32]:
T = 0.1
N = 5
M = 5

Явная разностная схема

In [33]:
def U(x, t):
    return x ^ 3 + t ^ 3


def f(x, t):
    tmp = 3 * t ** 2 - 18 * x - 9 * x ** 2 + x ** 4 + x * t ** 3
    return tmp


def phi(x):
    return x ** 3


def alpha(t):
    return 0


def beta(t):
    return 1 + t ** 3


def p(x):
    return x + 3


def c(x):
    return -x


def approximate_L(uk, k, i, N, M):
    h = 1 / N
    tau = T / M
    tmp1 = p((i + 0.5) * h) * (uk[i + 1] - uk[i]) / h ** 2
    tmp2 = p((i - 0.5) * h) * (uk[i] - uk[i - 1]) / h ** 2
    tmp3 = c(i * h) * uk[i]
    return tmp1 - tmp2 + tmp3


In [34]:
u = []
arr = [0] * (N + 1)
h = 1 / N
tau = T / M
for i in range(N + 1):
    xi = i * h
    arr[i] = phi(xi)
u.append(arr)  # U^0

for k in range(1, M + 1):

    arr = [0] * (N + 1)
    for i in range(1, N):
        xi = i * h
        t_k_1 = tau * (k - 1)
        arr[i] = u[k - 1][i] + tau * (approximate_L(u[k - 1], k - 1, i, N, M) + f(xi, t_k_1))
    arr[0] = (alpha(k * tau) * 2 * h + arr[2] - 4 * arr[1]) / (-3)
    arr[N] = beta(k * tau)
    u.append(arr)

x = [i * h for i in range(N + 1)]
t = [k * tau for k in range(M + 1)]
table = pd.DataFrame(u, index=t, columns=x)
table.columns.name = "t \\ x"
table

t \ x,0.0,0.2,0.4,0.6,0.8,1.0
0.00,0.000000,0.008000,0.064000,0.216000,0.512000,1.000000
0.02,-0.009867,0.008800,0.064800,0.216800,0.512800,1.000008
0.04,-0.031089,-0.006913,0.065618,0.217614,0.512067,1.000064
0.06,-0.018081,-0.003756,0.039221,0.215634,0.515820,1.000216
0.08,-0.080196,-0.033985,0.104649,0.181677,0.502007,1.000512
0.10,0.113328,0.044488,-0.162035,0.359489,0.478835,1.001000


Схема с весами.

$\sigma = 0$ - т.к. $\alpha_2 \neq 0$, 
аппроксимация производится явной схемой, описанной выше

$\sigma = 1$ - 

In [35]:
def solve(N, A, B, C, G):
    sol = np.zeros((N + 1, 1))
    s = np.zeros((N + 1, 1))
    t = np.zeros((N + 1, 1))

    s[0] = C[0] / B[0]
    t[0] = -G[0] / B[0]

    for i in range(1, N + 1):
        s[i] = C[i] / (B[i] - A[i] * s[i - 1])
        t[i] = (A[i] * t[i - 1] - G[i]) / (B[i] - A[i] * s[i - 1])
    sol[N] = t[N]
    for i in range(N - 1, -1, -1):
        sol[i] = sol[i + 1] * s[i] + t[i]
    return sol


def weights_scheme(sigma):
    if sigma == 1 / 2:
        coef = -1 / 2
    else:
        coef = 0

    u = []

    # U^0
    arr = [0] * (N + 1)
    for i in range(N + 1):
        arr[i] = phi(i * h)
    u.append(arr)

    for k in range(1, M + 1):
        A = np.zeros((N + 1, 1))
        B = np.zeros((N + 1, 1))
        C = np.zeros((N + 1, 1))
        G = np.zeros((N + 1, 1))

        B[0] = 1
        C[0] = 1
        G[0] = alpha(tau * k) * h

        A[N] = 0
        B[N] = -1
        G[N] = beta(tau * k)

        for i in range(1, N):
            G[i] = -u[k - 1][i] / tau - (1 - sigma) * approximate_L(u[k - 1], k, i, N, M) - f(i * h, tau * (k + coef))
            A[i] = p((i + 1 / 2) * h) / h ** 2
            B[i] = ((p((i + 1 / 2) * h) + p((i - 1 / 2) * h)) / h ** 2 - c(i * h)) + 1 / tau
            C[i] = p((i + 1 / 2) * h) / h ** 2

        arr = solve(N, A, B, C, G)
        u.append(arr)
    return u


In [36]:
print('sigma = 1')
data = weights_scheme(1)
x = [i * h for i in range(N + 1)]
t = [k * tau for k in range(M + 1)]
table = pd.DataFrame(data, index=t, columns=x)
table.columns.name = "t \\ x"
table

sigma = 1


t \ x,0.0,0.2,0.4,0.6,0.8,1.0
0.00,0,0.008,0.064,0.216,0.512,1
0.02,[0.018367091278563733],[0.018367091278563733],[0.07154758734875866],[0.2237161664547666],[0.5197836903211784],[1.000008]
0.04,[0.027668760452279458],[0.027668760452279458],[0.07961858601003516],[0.23038988399542126],[0.5241399061694871],[1.000064]
0.06,[0.03627295527109463],[0.03627295527109463],[0.08722636564248255],[0.2362673337243499],[0.527511184163615],[1.000216]
0.08,[0.044275952246338894],[0.044275952246338894],[0.09429683561579086],[0.24162918475476028],[0.5305379332299414],[1.000512]
0.10,[0.051781117818592085],[0.051781117818592085],[0.10093152888626987],[0.2466699357327717],[0.5334514758946939],[1.001]


In [37]:
print('sigma = 1/2')
data = weights_scheme(0.5)
x = [i * h for i in range(N + 1)]
t = [k * tau for k in range(M + 1)]
table = pd.DataFrame(data, index=t, columns=x)
table.columns.name = "t \\ x"
table

sigma = 1/2


t \ x,0.0,0.2,0.4,0.6,0.8,1.0
0.00,0,0.008,0.064,0.216,0.512,1
0.02,[0.09004032440068666],[0.09004032440068666],[0.15826726334506286],[0.32103401207076304],[0.6002769808010773],[1.000008]
0.04,[0.18135315068760188],[0.18135315068760188],[0.2400862851364482],[0.38251328021748593],[0.6287344042820132],[1.000064]
0.06,[0.25537351624353405],[0.25537351624353405],[0.30411889707846124],[0.42885412578523446],[0.6540338642845336],[1.000216]
0.08,[0.31410031479849165],[0.31410031479849165],[0.3551558324821126],[0.466292461293733],[0.6735045899301837],[1.000512]
0.10,[0.3609729878179294],[0.3609729878179294],[0.395915865835824],[0.4961037820854691],[0.6893936042580483],[1.001]
